In [1]:
%cd ..

/home/thomas/Documents/Tesis/StudentVisorBackend/server_pyflask


In [2]:
import pandas as pd
from src.transformations.common_operations import filterDataFrame
from src.utils.enums import ColName
from src.transformations.common_operations import readFile

In [31]:
filters1 = {}
filters1[ColName.INSC_TYPE.value] = "i"
filters2 = {}
filters2[ColName.INSC_TYPE.value] = "i"

schFilters = {}

table1 = readFile("../studentsdata/students_inscriptions/2021_student-inscriptions_1d6c7dee-e94f-44eb-b4a7-49ac3b738b95.pickle")
table2 = readFile("../studentsdata/students_inscriptions/2023_student-inscriptions_5bb4a8a7-4cec-48f2-94df-00c3c1db17a5.pickle")

scholarships = readFile("../studentsdata/students_scholarships/2023_student-scholarships-belgrano_e14a08dc-7b43-4feb-be17-26fc9376fa3a.pickle")

In [32]:
table1[ColName.ID.value] = table1[ColName.ID.value].astype(str)
table2[ColName.ID.value] = table2[ColName.ID.value].astype(str)

table1[ColName.OFFER.value] = table1[ColName.OFFER.value].astype(str)
table2[ColName.OFFER.value] = table2[ColName.OFFER.value].astype(str)

table1[ColName.UNIT.value] = table1[ColName.UNIT.value].astype(str)
table2[ColName.UNIT.value] = table2[ColName.UNIT.value].astype(str)

table1[ColName.INSC_TYPE.value] = table1[ColName.INSC_TYPE.value].astype(str)
table2[ColName.INSC_TYPE.value] = table2[ColName.INSC_TYPE.value].astype(str)

scholarships[ColName.OFFER.value] = scholarships[ColName.OFFER.value].astype(str)
scholarships[ColName.UNIT.value] = scholarships[ColName.UNIT.value].astype(str)
scholarships[ColName.ID.value] = scholarships[ColName.ID.value].astype(str)

In [33]:
print(table1.dtypes)
print(table2.dtypes)
print(scholarships.dtypes)

unit       object
offer      object
insc       object
sex      category
id         object
dtype: object
unit       object
offer      object
insc       object
sex      category
id         object
dtype: object
unit     object
offer    object
id       object
dtype: object


In [34]:
table1 = table1[table1['insc']=="i"]
table2 = table2[table2['insc']=="i"]

In [23]:
table1 = filterDataFrame(table1, filters1)
table2 = filterDataFrame(table2, filters2)
scholarships = filterDataFrame(scholarships, schFilters)

In [35]:
print(table1.dtypes)
print(table2.dtypes)
print(scholarships.dtypes)

unit       object
offer      object
insc       object
sex      category
id         object
dtype: object
unit       object
offer      object
insc       object
sex      category
id         object
dtype: object
unit     object
offer    object
id       object
dtype: object


In [17]:
activity = table1.merge(table2,on=ColName.ID.value,how='inner')

In [16]:
activity: pd.DataFrame = table1.merge(table2, on=[ColName.ID.value], how="inner")

In [19]:
# Conversion from categorical to string to allow comparison between series
activity[ColName.OFFER.value + "_x"] = activity[
    ColName.OFFER.value + "_x"
].astype(str)

activity[ColName.OFFER.value + "_y"] = activity[
    ColName.OFFER.value + "_y"
].astype(str)

In [20]:
aux11 = activity[ColName.OFFER.value + "_x"] != activity[ColName.OFFER.value + "_y"]

In [21]:
differentActivity = activity[activity[ColName.OFFER.value + "_x"] != activity[ColName.OFFER.value + "_y"]]

In [25]:

set(differentActivity['id']).intersection(set(scholarships['id']))



{'41071983', '43909013'}

In [22]:
differentActivityWithScholarships = differentActivity.merge(scholarships,on=ColName.ID.value,how="inner",)

In [13]:

differentActivityWithScholarships = differentActivityWithScholarships[
    differentActivityWithScholarships[ColName.OFFER.value + "_y"]
    == differentActivityWithScholarships[ColName.OFFER.value]
]

In [14]:
differentActivityWithScholarships = (
    differentActivityWithScholarships.drop_duplicates(subset=[ColName.ID.value])
    .groupby(ColName.OFFER.value, observed=True)[
        ColName.OFFER.value
    ]  # observed only applies if grouping by categorical columns,but set it still.
    .count()
)

In [15]:
differentActivityWithScholarships

offer
tecnicatura universitaria en desarrollo de aplicaciones informaticas    1
Name: offer, dtype: int64